In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

# # Download necessary NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# # Initialize a Porter stemmer
# stemmer = PorterStemmer()

from datasets import DatasetDict


In [2]:
nltk.download('punkt')
# nltk.load('wordnet')


[nltk_data] Error loading punkt: <urlopen error [Errno 54] Connection
[nltk_data]     reset by peer>


False

In [3]:
import pandas as pd
import re

In [37]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")

No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (/Users/ashnadua/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
def preprocess(example):
    document_article = example['article']
    # print(document_article)

    sentences = document_article.split(" .")

    sent_articles = ""
    for sen in sentences:
      sen = sen.replace('\n', ' ')
      sen = ' '.join(word for word in sen.split() if not word.startswith('@'))
      sen = re.sub(r'\W+', ' ', sen)
      sent_articles += sen
      sent_articles += ". "
    document_abstract = example['abstract']
    sentences = document_abstract.split(" .")
    sent_abstract = ""
    for sen in sentences:
      sen = sen.replace('\n', ' ')
      sen = ' '.join(word for word in sen.split() if not word.startswith('@'))
      sen = re.sub(r'\W+', ' ', sen)
      sent_abstract += sen
      sent_abstract += ". "

    return {'article': sent_articles, 'abstract': sent_abstract}

In [39]:
dataset_send = DatasetDict()

In [40]:
dataset_send["train"] = dataset["train"].select(range(1000))
dataset_send["validation"] = dataset["validation"].select(range(1000))
dataset_send["test"] = dataset["test"].select(range(1000))

In [41]:
dataset["train"] = dataset["train"].select(range(500))
dataset["validation"] = dataset["validation"].select(range(500))
dataset["test"] = dataset["test"].select(range(500))

In [42]:
dataset["train"] = dataset["train"].map(preprocess)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [43]:
dataset["validation"] = dataset["validation"].map(preprocess)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [44]:
dataset["test"] = dataset["test"].map(preprocess)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [45]:
df_train = dataset["train"].to_pandas()

In [89]:
dataset_send["test"] = dataset_send["test"].map(preprocess)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import math


In [51]:
cue_phrases = ["the paper describes", "in conclusion", "in summary", "our investigation", "the best", "the most important", "in particular", "according to the study", "significantly", "important", "hardly", "impossible"]

def calculate_scores(articles):
    scores = []
    tfidf_vectorizer = TfidfVectorizer()
    
    # Split each article into sentences and flatten the list
    sentences = [sentence for article in articles for sentence in article.split('. ')]
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    for i, sentence in enumerate(sentences):
        sLen = len(word_tokenize(sentence))
        sPos = i / len(sentences)

        pos_tags = nltk.pos_tag(word_tokenize(sentence))
        nvpi = len([word for word, pos in pos_tags if pos in ['NN', 'NNS', 'VB', 'VBD', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']])
        PNi = len([word for word, pos in pos_tags if pos in ['NNP', 'NNPS']])

        current_tfidf = tfidf_matrix[i].toarray()
        acs = cosine_similarity(current_tfidf, tfidf_matrix).mean()
        cp_count = sum(1 for phrase in cue_phrases if phrase in sentence.lower())
        tf_idf = current_tfidf.sum()

        scores.append({
            "sentence": sentence,
            "F1": sLen,
            "F2": sPos,
            "F3": tf_idf,
            "F4": nvpi,
            "F5": PNi,               
            "F6": acs,
            "F7": cp_count,
        })

    return scores


In [90]:
def calculate_scores_for_each_sentence(articles):
    all_scores = []
    
    for article in articles:
        # Split the article into sentences
        sentences = article.split('. ')
        
        scores = calculate_scores(sentences)
        
        all_scores.append(scores)
    
    return all_scores


In [92]:
all_scores = calculate_scores_for_each_sentence(dataset_send["test"]["article"])

In [96]:
len(all_scores[0])

247

In [52]:
# scores = calculate_scores(df_train["article"])


In [54]:
# def restructure_scores(articles, scores):
#     start = 0
#     all_scores = []
#     for article in articles:
#         num_sentences = len(article.split(". "))
#         article_scores = scores[start:start+num_sentences]
#         all_scores.append(article_scores)
#         start += num_sentences
#     return all_scores

In [55]:
# grouped_scores = restructure_scores(df_train["article"], scores)

In [99]:
def normalize_sentence_length(sLen):
    sLenmax = max(sLen)
    if sLenmax == 0:
        return [0.0] * len(sLen)  # Avoid division by zero
    return [sLeni / sLenmax for sLeni in sLen]

def normalize_tfidf(tfidf_scores):
    tfidf_max = max(tfidf_scores)
    if tfidf_max == 0:
        return [0.0] * len(tfidf_scores)  # Avoid division by zero
    return [(tfidf_i / tfidf_max) for tfidf_i in tfidf_scores]

def normalize_noun_verb_phrase(nvpi):
    nvpi_max = max(nvpi)
    if nvpi_max == 0:
        return [0.0] * len(nvpi)  # Avoid division by zero
    return [(nvpi_i / nvpi_max) for nvpi_i in nvpi]

def normalize_proper_noun(PNi):
    PNmax = max(PNi)
    if PNmax == 0:
        return [0.0] * len(PNi)  # Avoid division by zero
    return [(PNi_i / PNmax) for PNi_i in PNi]

def normalize_cosine_similarity(ACS):
    ACSmax = max(ACS)
    if ACSmax == 0:
        return [0.0] * len(ACS)  # Avoid division by zero
    return [(ACS_i / ACSmax) for ACS_i in ACS]

def normalize_cue_phrases(CPi, tCP):
    if tCP == 0:
        return [0.0] * len(CPi)  # Avoid division by zero
    return [(CPi_i / tCP) for CPi_i in CPi]


In [100]:
def normalize_scores(grouped_scores):
    normalized_scores = []
    for scores in grouped_scores:
        df = pd.DataFrame(scores)
        
        df['F1'] = normalize_sentence_length(df['F1'])
        df['F3'] = normalize_tfidf(df['F3'])
        df['F4'] = normalize_noun_verb_phrase(df['F4'])
        df['F5'] = normalize_proper_noun(df['F5'])
        df['F6'] = normalize_cosine_similarity(df['F6'])
        df['F7'] = normalize_cue_phrases(df['F7'], df['F7'].sum())
        
        normalized_scores.append(df)
    
    return normalized_scores

In [101]:
normalized_scores = normalize_scores(all_scores)


In [103]:
for df in normalized_scores:
    df['total_score'] = df[['F1', 'F3', 'F4', 'F5', 'F6', 'F7']].sum(axis=1)
    df.sort_values(by='total_score', ascending=False, inplace=True)


In [75]:
import numpy as np

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

def generate_summary(normalized_scores, num_sentences=3, similarity_threshold=0.8):
    summaries = []
    
    for df in normalized_scores:
        # Sort the DataFrame by 'total_score' in descending order and reset the index
        df_sorted = df.sort_values(by='total_score', ascending=False).reset_index(drop=True)
        
        # Fit the TF-IDF vectorizer on the sentences of the current article
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_vectorizer.fit(df_sorted['sentence'])
        
        summary_sentences = []
        for _, row in df_sorted.iterrows():
            sentence = row['sentence']
            if not summary_sentences:
                summary_sentences.append(sentence)
            else:
                # Calculate cosine similarity with existing summary
                similarity = cosine_similarity(
                    tfidf_vectorizer.transform([sentence]),
                    tfidf_vectorizer.transform(summary_sentences)
                )
                # If the sentence is not too similar to the existing summary, add it
                if np.max(similarity) < similarity_threshold:
                    summary_sentences.append(sentence)
            # Stop adding sentences if we've reached the desired length
            if len(summary_sentences) == num_sentences:
                break
        
        # Join the sentences together into a summary
        summary = '. '.join(summary_sentences)
        
        summaries.append(summary)
    
    return summaries

# Now you can call the function like this:
summaries = generate_summary(normalized_scores, num_sentences=10, similarity_threshold=0.8)


In [114]:
dataset_send["test"] = dataset_send["test"].add_column("summary", summaries)


In [116]:
dataset_send["test"]

Dataset({
    features: ['article', 'abstract', 'summary'],
    num_rows: 1000
})

In [117]:
from rouge_score import rouge_scorer

class Score:
    def __init__(self, trueSummary = None, predSummary = None):
        self.trueSummary = trueSummary if trueSummary is not None else ""
        self.predSummary = predSummary if predSummary is not None else ""

    def rougeScore(self):
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3'], use_stemmer=True)
        scores = scorer.score(self.trueSummary, self.predSummary)

        return scores

In [119]:
dataset_send["test"][0]["abstract"]

'the short term periodicities of the daily sunspot area fluctuations from august 1923 to october 1933 are discussed. for these data the correlative analysis indicates negative correlation for the periodicity of about days but the power spectrum analysis indicates a statistically significant peak in this time interval. a new method of the diagnosis of an echo effect in spectrum is proposed and it is stated that the 155 day periodicity is a harmonic of the periodicities from the interval of days. the autocorrelation functions for the daily sunspot area fluctuations and for the fluctuations of the one rotation time interval in the northern hemisphere separately for the whole solar cycle 16 and for the maximum activity period of this cycle do not show differences especially in the interval of days. it proves against the thesis of the existence of strong positive fluctuations of the about interval in the maximum activity period of the solar cycle 16 in the northern hemisphere. however a sim

In [120]:
rouge_scores = []

for i in range(len(dataset_send["test"])):
    true_summary = dataset_send["test"][i]["abstract"]
    pred_summary = dataset_send["test"][i]["summary"]

    score_obj = Score(true_summary, pred_summary)
    scores = score_obj.rougeScore()
    rouge_scores.append(scores)

In [121]:
dataset_send["test"] = dataset_send["test"].add_column("Rouge", rouge_scores)


In [127]:
dataset_send["test"] = dataset_send["test"].rename_column('abstract', 'Gold Summary')
dataset_send["test"] = dataset_send["test"].rename_column('summary', 'Generated Summary')


In [128]:
dataset_send["test"]

Dataset({
    features: ['article', 'Gold Summary', 'Generated Summary', 'Rouge'],
    num_rows: 1000
})

In [130]:
dataset_send["test"].to_csv('sum_it_up_test.csv', index=False)


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

33101782